## CODE

In [8]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("open_api_key")

### 검색 증강 생성 개요

In [ ]:
# 코사인 유사도
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos_sim(A,B):
    return dot(A,B)/(norm(A)*norm(B))

vec1 = np.array([0,1,1,1])
vec2 = np.array([1,0,2,1])
vec3 = np.array([2,0,4,2])

print(f"벡터1과 벡터2의 유사도 : {cos_sim(vec1,vec2)}")
print(f"벡터2과 벡터3의 유사도 : {cos_sim(vec2,vec3)}")

In [ ]:
# OpenAI 임베딩 모델
import os
import numpy as np
from numpy import dot
from numpy.linalg import norm
import pandas as pd

from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import OpenAI

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=api_key)
query_result = embeddings.embed_query('저는 배가 고파요')
print(query_result)

In [ ]:
data = [
    '주식 시장이 급등했어요',
    '시장 물가가 올랐어요',
    '전통 시장에는 다양한 물품들을 팔아요',
    '저는 빠른 비트를 좋아해요',
    '최근 비트코인 가격이 많이 반등했어요',
]

df = pd.DataFrame(data,columns=['text'])
# print(df)

# 텍스트 -> 임베딩 벡터 변환 함수
def get_embedding(text):
    return embeddings.embed_query(text)

df['embeddings'] = df.apply(
    lambda row : get_embedding(row.text),
    axis=1
)
# print(df)

# 코사인 유사도 계산 함수
def cos_sim(A,B):
    return dot(A,B)/(norm(A)*norm(B))


def return_answer_candidate(df,query):
    query_embedding = get_embedding(query)

    df['similarity'] = df.embeddings.apply(lambda x : cos_sim(np.array(x), np.array(query_embedding)))

    top_three_doc = df.sort_values("similarity", ascending=False).head(3)

    return top_three_doc

sim_result = return_answer_candidate(df,'과일 값이 비싸다')
print(sim_result)


In [2]:
# 허깅페이스 제공 임베딩 모델
#from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import HuggingFaceBgeEmbeddings
from sentence_transformers import SentenceTransformer
import numpy as np
from numpy import dot
from numpy.linalg import norm
import pandas as pd


# 텍스트 -> 임베딩 벡터 변환 함수
def get_embedding(text):
    return embeddings.embed_query(text)

# 코사인 유사도 계산 함수
def cos_sim(A,B):
    return dot(A,B)/(norm(A)*norm(B))


def return_answer_candidate(df,query):
    query_embedding = get_embedding(query)

    df['similarity'] = df.embeddings.apply(lambda x : cos_sim(np.array(x), np.array(query_embedding)))

    top_three_doc = df.sort_values("similarity", ascending=False).head(3)

    return top_three_doc

embeddings = SentenceTransformer('BAAI/bge-m3')
embeddings = HuggingFaceBgeEmbeddings(model_name = 'BAAI/bge-m3')
# embeddings = HuggingFaceEmbeddings(model_name = 'BAAI/bge-m3')

data = [
    '주식 시장이 급등했어요',
    '시장 물가가 올랐어요',
    '전통 시장에는 다양한 물품들을 팔아요',
    '저는 빠른 비트를 좋아해요',
    '최근 비트코인 가격이 많이 반등했어요',
]

hugging_df = pd.DataFrame(data, columns=['text'])
hugging_df['embeddings'] = hugging_df['text'].apply(get_embedding)
# print(df)


sim_result = return_answer_candidate(hugging_df,'과일 값이 비싸다')
print(sim_result)


C:\Users\kimji\AppData\Local\Temp\ipykernel_39312\2227364096.py:30: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(model_name = 'BAAI/bge-m3')


                   text                                         embeddings  \
1           시장 물가가 올랐어요  [0.013636118732392788, 0.05754704773426056, -0...   
4  최근 비트코인 가격이 많이 반등했어요  [0.01619962975382805, 0.036948565393686295, -0...   
2  전통 시장에는 다양한 물품들을 팔아요  [0.01703060232102871, 0.04437505826354027, -0....   

   similarity  
1    0.702341  
4    0.673596  
2    0.667758  


### 문서로더

In [ ]:
import os
os.environ["USER_AGENT"] = "MyApp/1.0 (Custom Langchain Application)"

from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://docs.smith.langchain.com/")

loader_multiple_pages = WebBaseLoader(
    ["https://python.langchain.com/docs/introduction/",
     "https://langchain-ai.github.io/langgraph"]
)

single_doc = loader.load()
print(single_doc[0].metadata)

docs = loader_multiple_pages.load()
print(docs[0].page_content)

실습 pdf 파일 다운 : https://www.kbfg.com/kbresearch/report/reportView.do?reportId=2000450

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import PDFPlumberLoader

loader = PyPDFLoader(r"pdf주소")
pages = loader.load_and_split()
print(f"청크의 수 : {len(pages)}")
print(pages[10])

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(r"pdf주소")
pages = loader.load_and_split()
print(f"청크의 수 : {len(pages)}")
print(pages[10])

In [ ]:
from langchain_community.document_loaders import PDFPlumberLoader

loader = PDFPlumberLoader(r"C:\Users\kimji\Desktop\ProgramFile\Study\2024 KB 부동산 보고서_최종.pdf")
pages = loader.load_and_split()
print(f"청크의 수 : {len(pages)}")
print(pages[10])

In [ ]:
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader(r"CSV파일 경로")
documents = loader.load()
print(f"청크의 수 : {len(pages)}")
print(documents[5])

In [ ]:
from langchain_community.document_loaders import UnstructuredCSVLoader

loader = UnstructuredCSVLoader(r"CSV파일 경로")
documents = loader.load()
print(f"청크의 수 : {len(pages)}")
print(str(documents[0].metadata)[:500])
print(str(documents[0].page_content)[:500])

### 텍스트 분할

In [ ]:
# RecursiveCharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

url = r"C:\Users\kimji\Desktop\ProgramFile\Study\2024 KB 부동산 보고서_최종.pdf"

loader = PyPDFLoader(url)
pages = loader.load()
print(f"총 글자 수 : {len(''.join([i.page_content for i in pages]))}")

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap=50)
texts = text_splitter.split_documents(pages) # 단순히 긴 문자열 (텍스트파일) 분할 : .split_text()
print(f"분할된 청크의 수 : {len(texts)}")

print(texts[1])
print(texts[1].page_content)
print(texts[2].page_content)
print(len(texts[1].page_content))
print(len(texts[2].page_content))

In [9]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(url)
pages = loader.load()

text_splitter = SemanticChunker(embeddings=OpenAIEmbeddings(api_key=api_key))
chunks = text_splitter.split_documents(pages)
print(f"분할된 청크의 수 : {len(chunks)}")


분할된 청크의 수 : 164


In [ ]:
print(chunks[3])
print(chunks[4])
print(chunks[5])

In [11]:
text_splitter = SemanticChunker(
	OpenAIEmbeddings(api_key=api_key),
	breakpoint_threshold_type="standard_deviation",
	breakpoint_threshold_amount=3,
)
chunks = text_splitter.split_documents(pages)
print(f"분할된 청크의 수 : {len(chunks)}")

분할된 청크의 수 : 84


In [12]:
text_splitter = SemanticChunker(
	OpenAIEmbeddings(api_key=api_key),
	breakpoint_threshold_type="interquartile",
	breakpoint_threshold_amount=1.5,
)
chunks = text_splitter.split_documents(pages)
print(f"분할된 청크의 수 : {len(chunks)}")

분할된 청크의 수 : 142
